License

Copyright 2024 Hossein Askari

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

requirements (use  python below 3.10 if you want to train the model and want tp use CUDA)

In [ ]:
!pip install transformers pandas torch 

In [ ]:
## for checking gpu and cuda working properly
import torch
torch.cuda.is_available()
!nvidia-smi

function for cleaning the dataset from numbers and urls

In [ ]:
import re

def remove_mentions(text):
    mention_pattern = r'@\S+\s'
    clean_text = re.sub(mention_pattern, ' ', text)
    return clean_text


# Path to  original dataset
input_file_path = './tweets.txt'
# Path to the output file 
output_file_path = './cleaned_data.txt'

with open(input_file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

cleaned_lines = [remove_mentions(line) for line in lines]

with open(output_file_path, 'w', encoding='utf-8') as file:
    for line in cleaned_lines:
        file.write(line)


function for fine tuning GPT2 on our new  dataset

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Preparing dataset
dataset = TextDataset(
  tokenizer=tokenizer,
  file_path="INSERT FILE PATH HERE",
  block_size=128
)

data_collator = DataCollatorForLanguageModeling(
  tokenizer=tokenizer, mlm=False
)

# Training settings
training_args = TrainingArguments(
  output_dir="./finetuned",
  overwrite_output_dir=True,
  num_train_epochs=4,
  per_device_train_batch_size=4,
  save_steps=13000,  # Save the model after every n steps
  save_total_limit=2,  #  last 2 checkpoints will be kept
)

trainer = Trainer(
  model=model,
  args=training_args,
  data_collator=data_collator,
  train_dataset=dataset,
)

# Start 
trainer.train()

# Save the final model and tokenizer 
model.save_pretrained('./finetuned')
tokenizer.save_pretrained('./finetuned')


if the GPT still has restrictions we need to fine tune it once more using LLm-Attack method 
https://github.com/llm-attacks/llm-attacks/tree/main

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from torch.utils.data import Dataset
no_deprecation_warning=True
# Load the dataset
df = pd.read_csv("./harmful_behaviors.csv")

# Initializng the tokenizer and set the pad token
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Tokenizing the 'goal' and 'target' columns
max_length = 512  
input_encodings = tokenizer(df['goal'].tolist(), padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
target_encodings = tokenizer(df['target'].tolist(), padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")

# Creating a custom dataset
class TextDataset(Dataset):
    def __init__(self, input_encodings, target_encodings):
        self.input_encodings = input_encodings
        self.target_encodings = target_encodings

    def __len__(self):
        return len(self.input_encodings['input_ids'])

    def __getitem__(self, idx):
        input_ids = self.input_encodings['input_ids'][idx]
        attention_mask = self.input_encodings['attention_mask'][idx]
        labels = self.target_encodings['input_ids'][idx]
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

dataset = TextDataset(input_encodings, target_encodings)

# Initializing the model
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.config.pad_token_id = model.config.eos_token_id

#  Defining training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=4,              # number of training epochs
    per_device_train_batch_size=4,   # Batch size per device during training
    logging_dir='./logs',            # Directory for  logs
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Start training
trainer.train()

#Save the fine-tuned model
model.save_pretrained("./dob")
tokenizer.save_pretrained("./dob")


Run the fine tuned GPT

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline


# Load fine-tuned model
model = GPT2LMHeadModel.from_pretrained("./finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("./finetuned")

#  generation pipeline
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generating text (change max length for, well, more length)
generated_text = text_generator("how are you ", max_length=50)[0]['generated_text']

print(generated_text)

############################

codes for text to speech 

############################

function to turn csv dataset to text file with one empty line between them (for audio generation)

In [ ]:
import csv

def csv_column_to_text_with_separator(input_csv_path, column_name, output_text_path):

    with open(input_csv_path, mode='r', newline='', encoding='utf-8') as csv_file:
        # Initialize the CSV reader
        reader = csv.DictReader(csv_file)
        
        with open(output_text_path, mode='w', encoding='utf-8') as text_file:
            # Write each value from the specified column to the text file, followed by an empty line
            for row in reader:
                if column_name in row:
                    text_file.write(row[column_name] + '\n\n')
                else:
                    print(f"Column '{column_name}' not found in the CSV file.")

input_csv = './hate.csv'
column_name = 'text'
output_text = 'new.txt'
csv_column_to_text_with_separator(input_csv, column_name, output_text)

code for selecting random lines (for audio generation)

In [ ]:
import random

def get_random_lines_with_number_at_end(filename, num_lines=5, start_number=1):

    try:
        with open(filename, 'r', encoding='utf-8') as file:
            lines = file.readlines()
        
        if num_lines > len(lines):
            print(f"File has only {len(lines)} lines. Adjusting num_lines to {len(lines)}.")
            num_lines = len(lines)
        
        random_lines = random.sample(lines, num_lines)
        
        for i, line in enumerate(random_lines, start=start_number):
            print(f"{line.strip()} ")
    except FileNotFoundError:
        print(f"File '{filename}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


get_random_lines_with_number_at_end('./tweets.txt', num_lines=5, start_number=1)
